In [1]:
pip install unidecode

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import mysql.connector
from unidecode import unidecode

In [3]:
database = "ofertas_de_empleo"
db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "Scarface",
                             database = database)

query_numero = """
SELECT 
    o.id_oferta,
    COUNT(DISTINCT tr.tec_id) AS num_tecnologias,
    COUNT(DISTINCT hr.hab_id) AS num_habilidades
FROM 
    ofertas o
LEFT JOIN 
    tecnologias_relacion tr ON o.id_oferta = tr.id_oferta
LEFT JOIN 
    habilidades_relacion hr ON o.id_oferta = hr.id_oferta
GROUP BY 
    o.id_oferta;
"""

df_sql = pd.read_sql(query_numero, db)
df_sql.to_csv("df_sql.csv", index=False)


/tmp/ipykernel_28967/4129477080.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query_numero, db)


In [4]:
df_sql.rename(columns={"id_oferta": "id"}, inplace=True)

In [5]:
df_general = pd.read_csv("../ofertas_final.csv")

In [6]:
##pickle - encoder

df_general_codificado = pd.DataFrame()
df_general_codificado["id"] = df_general["id"]
df_general_codificado["salario_media"] = (df_general["salario_desde"] + df_general["salario_hasta"]) / 2
df_general_codificado["variable"] = df_general["variable"].fillna(0)
df_general_codificado["variable_booleano"] = df_general_codificado["variable"].apply(lambda x: 1 if x != 0 else 0)
moda_vacaciones = df_general["vacaciones"].mode()[0]
df_general_codificado["vacaciones"] = df_general["vacaciones"].fillna(moda_vacaciones) 
df_general_codificado["teletrabajo"] = df_general["teletrabajo"].map({"Presencial": 0, "Hibrido": 1, "Remoto": 2})
df_general_codificado["jornada_tipo"] = df_general["jornada_tipo"].map({"Intensiva Mañana": 0, "Intensiva Tarde": 1, "Jornada Completa": 2, 
                                                                "Jornada Completa Flexible": 3, "Media Jornada": 4, "Por Horas": 5, "Turno Rotatorio" : 6})

categorias_funciones = {cat: i for i, cat in enumerate(df_general["funciones"].dropna().unique())}
df_general_codificado["funciones"] = df_general["funciones"].map(categorias_funciones)
df_general_codificado["contrato_tipo"] = df_general["contrato_tipo"].map({"Indefinido": 0, "Temporal": 1, "Obra o servicio": 2,
                                                                  "Freelance/Autónomo": 3, "Prácticas": 4, "A determinar": 5})
df_general_codificado["nivel_profesional"] = df_general["nivel_profesional"].map({"Administrador / Consejero": 0, "Director / Gerente": 1, 
                                                                          "Mando Intermedio": 2, "Especialista": 3, "Empleado": 4, "Prácticas / Beca": 5})
df_general_codificado["formacion_minima"] = df_general["formacion_minima"].map({"Doctorado": 0, "Ingeniero Superior": 1, "Ingeniero Técnico": 2, "Licenciado": 3, 
                                                                        "Postgrado EEES (Máster)": 4, "Diplomado": 5, "FP2/Grado Superior": 6, 
                                                                        "Grado EEES (Bolonia)": 7, "Grado Medio": 8, "Bachillerato/COU": 9, 
                                                                        "E.S.O. (Educación Secundaria Obligatoria)": 10, "FP1": 11, "Sin estudios": 12, 
                                                                        "Otros títulos, certificaciones y carnets": 13, "Certificado de Profesionalidad": 14, 
                                                                        "Otra Formación Tecnológica": 15})

df_general_codificado["personas_a_cargo"] = df_general["personas_a_cargo"].fillna(0)
df_general_codificado["experiencia"] = df_general["experiencia"].apply(lambda x: 0 if x == "Sin Experiencia" 
                                                               else 1 if x in ["Menos de un año", "1 año"]
                                                               else 2 if x in ["2 años", "3 años", "3-5 años"]
                                                               else 3)

In [7]:
df_codificado = pd.merge(df_general_codificado, df_sql, on='id', how='left')

In [8]:
imputer = KNNImputer(n_neighbors=5)
columnas = df_codificado.drop(columns=["id"])
df_columnas_imputadas = imputer.fit_transform(columnas)
df_columnas_imputadas = pd.DataFrame(df_columnas_imputadas, columns=columnas.columns)
df_imputado = pd.concat([df_columnas_imputadas, df_codificado["id"]], axis=1)
df_imputado.to_csv("/home/bosser/Escritorio/proyectofinal/df_imputado.csv", index = False)

In [9]:
##para la segunda parte dbscan